# Advance Business Constraints in MIP, LP

In [32]:
import pandas as pd
import numpy as np

import gurobipy as gb
from gurobipy import *

# Problem 1: BalancedMilk and the Pollution Tax (35 points)

## Problem Description
By now, you know BalancedMilk's story and its problems by heart! In Problem 2.3 of the previous quiz, you solved the problem where BalancedMilk wanted to optimize its transportation costs and pollution (CO2 emission). In this problem, we want to model situations where BalancedMilk plans its milk distribution considering different forms of pollution tax.

In the setting for this problem, $S_8$ is active and available. Therefore, all supply centers are available to supply milk to each demand market. **Also, in this problem, we impose the constraint that the total milk delivered to a demand market must be equal to its demand - neither more nor less.** If you pay attention to the Supply vector, you can see that the total supply is now more than the total demand. Nevertheless, logically, **no supply center can supply more than its capacity.**

## Data:
In the cell for each question, you can find the transportation costs ($/tonne) and the pollution (Kg/tonne) between each supply center and demand market.

## Question 1: The Simplest Case (5 points)

In Question 8 of Quiz 1, we introduced two objective functions and solved a multi-objective problem. In this question, however, we will solve a single-objective problem. In the following way, we can implement the weighted sum approach with a single objective instead of two separate objective functions. With a **t** $/Kg pollution tax, we can write the objective function as follows:

$$Total Cost = Transportation Cost + t \times Pollutions$$

In this equation, $Transportation Cost$ shows the total transportation costs, and $Pollutions$ shows the total pollution in Kg.

Now, set $t = 3.5 \$/Kg$ and write the code for the objective function to minimize the Total Cost. **Print the total transportation cost, total pollution (in Kg), and the total cost.**

In [33]:
Pollution = [[2, 4, 4, 5, 4, 5, 6, 2, 8, 2],
             [3, 10, 3, 4, 2.5, 2, 9, 1, 2, 5],
             [3, 4, 8, 3, 2, 1.5, 7, 5, 3, 8],
             [10, 5, 2, 5, 3, 5, 3, 6, 2, 7],
             [1, 1, 4, 5, 5, 2, 6, 2, 4, 3],
             [2, 4, 3, 4.5, 2, 7, 3, 3, 6, 1],
             [6, 7, 3, 3, 3, 6, 4, 6, 3, 10],
             [3, 3, 1, 6, 1, 5, 6, 3, 2, 4]]

Cost = [[35, 49, 16, 30,  15, 35, 12, 25, 10, 12],
        [15, 53, 7, 45, 47, 11, 16, 17, 15, 9],
        [22, 25, 42, 22, 31, 9, 11, 29, 36, 5],
        [32,  6, 40, 35, 49, 25, 30, 47, 32, 12],
        [9, 12, 19, 17, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 22, 5, 47, 30, 23, 19, 8],
        [43, 19,  5, 28, 47, 39, 15, 12, 9, 51],
        [34, 34, 10, 21, 9, 12, 14, 8, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [120, 100, 100, 260, 120, 250, 150, 500]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [100, 75, 150, 150, 180, 215, 170, 90, 160, 90]


n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)

t = 3.5

##############################
# Initialize the model:
model = gb.Model("Question1")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Create the decision variables:
x = model.addVars(n_supply, n_demand, vtype=GRB.CONTINUOUS, name='x')

#############################################################
# Your code here:
# Introduce the objective function:

transportation = quicksum(Cost[i][j]*x[i,j] for i in Range_supply for j in Range_demand)
Pollution_cost = t * quicksum(Pollution[i][j]*x[i,j] for i in Range_supply for j in Range_demand)

model.setObjective( transportation + Pollution_cost , GRB.MINIMIZE)




#############################################################
# Add the constraints:
model.addConstrs(quicksum(x[i,j] for j in Range_demand) <= Supply[i] for i in Range_supply)
model.addConstrs(quicksum(x[i,j] for i in Range_supply) == Demand[j] for j in Range_demand)



# Optimize the model:
model.optimize()


#Print the optimal values of the objective functions:
# print(f'Total Pollution = {Pollution.getValue()} Kgs.')
# print('Total Transport Cost = $%g.' % sum(Cost[i][j]*x[i,j].x for i in Range_supply for j in Range_demand))
# print('Total Cost = $%g.' % model.objVal)

# Print the optimal values of the objective functions:
print('Total Pollution = %g Kgs.' % sum(Pollution[i][j]*x[i,j].x for i in Range_supply for j in Range_demand))
print('Total Transport Cost = $%g.' % sum(Cost[i][j]*x[i,j].x for i in Range_supply for j in Range_demand))
print('Total Cost = $%g.' % model.objVal)


Total Pollution = 3700 Kgs.
Total Transport Cost = $15090.
Total Cost = $28040.


## Question 2: Adding some Constraints

Now, we want to add some constraints to the previous question. The setting is the same as the previous question: $t = 3.5 \$/Kg$, $S_8$ is active, the total milk delivered to each demand market must be equal to its demand, no supply center can supply more than its capacity, and the objective function is the same as the previous question.

We define the *pollution caused by delivering milk to demand market j* as follows:

$$
Pol(j) = \sum_{i = 1}^8 p_{i,j} x_{i,j},
$$

where $p_{i,j}$ is the pollution (Kg/tonne) caused by carrying a tonne of milk between supply center $i$ and demand market $j$, and $x_{i,j}$ is the amount of milk (in tonnes) delivered from supply center $i$ to demand market $j$.

Now, we want to add the following constraint to the previous question:

1. Total pollution should be less than or equal to 3000 Kg.

2. Total transportation cost should be less than or equal to $25000.

3. At most 5 demand markets can have $Pol(j) > 150$ Kg.

### Question 2.1: (10 points)
Write the mathematical formulation for each of the additional constraints:

### Constraint 1: Total pollution should be less than or equal to 3000 Kg:

sum(Pol[j] for j in Range_demand) <= 3000
...

### Constraint 2: Total transportation cost should be less than or equal to $25000:

quicksum(Cost[i][j]*x[i,j] for i in Range_supply for j in Range_demand) <=25000

...

### Constraint 3: At most 5 demand markets can have $Pol(j) > 150$ Kg:

add binary variables wi, for j in Range_demand

for j in Range_demand:
    model.addConstr(Pol[j] >= 150 + 0.000000000000001 M - w[j].M)

model.addConstr(sum(w[j] for j in in Range_demand) <= 5)

...


### Question 2.2: (5 points)

Write the code for Question 2, and **print the total transportation cost, total pollution (in kg), and the total cost.**

In [34]:
Pollution = [[2, 4, 4, 5, 4, 5, 6, 2, 8, 2],
             [3, 10, 3, 4, 2.5, 2, 9, 1, 2, 5],
             [3, 4, 8, 3, 2, 1.5, 7, 5, 3, 8],
             [10, 5, 2, 5, 3, 5, 3, 6, 2, 7],
             [1, 1, 4, 5, 5, 2, 6, 2, 4, 3],
             [2, 4, 3, 4.5, 2, 7, 3, 3, 6, 1],
             [6, 7, 3, 3, 3, 6, 4, 6, 3, 10],
             [3, 3, 1, 6, 1, 5, 6, 3, 2, 4]]

Cost = [[35, 49, 16, 30,  15, 35, 12, 25, 10, 12],
        [15, 53, 7, 45, 47, 11, 16, 17, 15, 9],
        [22, 25, 42, 22, 31, 9, 11, 29, 36, 5],
        [32,  6, 40, 35, 49, 25, 30, 47, 32, 12],
        [9, 12, 19, 17, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 22, 5, 47, 30, 23, 19, 8],
        [43, 19,  5, 28, 47, 39, 15, 12, 9, 51],
        [34, 34, 10, 21, 9, 12, 14, 8, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [120, 100, 100, 260, 120, 250, 150, 500]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [100, 75, 150, 150, 180, 215, 170, 90, 160, 90]


n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)

t = 3.5

##############################
# Initialize the model:
model = gb.Model("Question2")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Your code goes here:

# Create the decision variables:
x = model.addVars(n_supply, n_demand, vtype=GRB.CONTINUOUS, name='x')

transportation = quicksum(Cost[i][j]*x[i,j] for i in Range_supply for j in Range_demand)
Pollution_Cost = t * quicksum(Pollution[i][j]*x[i,j] for i in Range_supply for j in Range_demand)


Pol = model.addVars(Range_demand, lb = 0, vtype = 'C', name = ['pol'+str(j) for j in Range_demand])
model.addConstrs(Pol[j] == quicksum(Pollution[i][j]*x[i,j] for i in Range_supply) for j in Range_demand)

model.setObjective( transportation + Pollution_Cost , GRB.MINIMIZE)



#############################################################
# Add the constraints:
model.addConstrs(quicksum(x[i,j] for j in Range_demand) <= Supply[i] for i in Range_supply)
model.addConstrs(quicksum(x[i,j] for i in Range_supply) == Demand[j] for j in Range_demand)

# constraint 1
model.addConstr(sum(Pol[j] for j in Range_demand) <= 3000)

# constraint 2
model.addConstr(quicksum(Cost[i][j]*x[i,j] for i in Range_supply for j in Range_demand) <=25000)

# constraint 3
W = model.addVars(Range_demand, lb = 0, vtype = 'B', name = ['w'+str(j) for j in Range_demand])
M = 100000000000

for j in Range_demand:
    model.addConstr(Pol[j] >= 150 + 0.000000000000001 - W[j]*M)

model.addConstr(sum(W[j] for j in Range_demand) <= 5)

# Optimize the model:
model.optimize()


# Print the optimal values of the objective functions:
print('Total Pollution = %g Kgs.' % sum(Pollution[i][j]*x[i,j].x for i in Range_supply for j in Range_demand))
print('Total Transport Cost = $%g.' % sum(Cost[i][j]*x[i,j].x for i in Range_supply for j in Range_demand))
print('Total Cost = $%g.' % model.objVal)

Total Pollution = 3000 Kgs.
Total Transport Cost = $18983.6.
Total Cost = $29483.6.


## Question 3: Variable Pollution Tax - The Interval Approach

In some taxation schemes, the pollution tax imposed by the government varies depending on the amount of pollution caused by BalancedMilk. One of the approaches in imposing a tax is the **interval taxation**. This approach has several intervals of total pollution, each with its own fixed tax. In the following table, we have the total tax for each interval. 

| Interval (Kg) | Total Tax (\$) |
| :--: | :--: |
| [0, 1000] | 0 |
| (1000, 1750] | 1,000 |
| (1750, 2500] | 5,000 |
| (2500, 3250] | 10,000 |
| (3250, 4000] | 20,000 |

For example, if $1000 Kg < Pollutions \leq 1750 Kg$, the total pollution tax is $1,000. As you can see, this tax is independent of the exact value of the pollution and only depends on which interval it is located.  

### Question 3.1: (10 points)

Using the technique you learned in the course, write the mathematical formulation for the total tax. That is, suppose the total pollution ($Pollutions$) is given to you, and there is a decision variable $Tax$. Write the mathematical formulation for $Tax$ that, depending on the value of $Pollutions$, gives you the correct tax for that interval. You may introduce a series of variables to help you with this formulation. However, you cannot use the conditional statement (if-else) in your formulation - only mixed integer linear programming ideas are allowed.

(Hint: Suppose a binary variable $y$ and a very large number $M = 100000$. The constraint 
$$Pollutions - 1000 \leq M.y$$ 
ensures $y = 1$ when $Pollutions > 1000$.)

*Your formulation goes here:*

# tax constraints
M = 1000000
S = 0.0001


model.addConstr(tax == Tax[0] * y[0] + Tax[1] * y[1] + Tax[2] * y[2] + Tax[3] * y[3] + Tax[4] * y[4])

model.addConstr(total_pollution >= Interval[0] + S - M * y[0])

model.addConstr(total_pollution >= Interval[1] - M * (1 - y[0]))

model.addConstr(total_pollution >= Interval[1] + S - M * y[1])

model.addConstr(total_pollution >= Interval[2] - M * (1 - y[1]))

model.addConstr(total_pollution >= Interval[2] + S - M * y[2])

model.addConstr(total_pollution >= Interval[3] - M * (1 - y[2]))

model.addConstr(total_pollution >= Interval[3] + S - M * y[3])

model.addConstr(total_pollution >= Interval[4] - M * (1 - y[3]))

model.addConstr(total_pollution >= Interval[4] + S - M * y[4])

model.addConstr(y[0] + y[1] + y[2] + y[3] + y[4] == 1)







### Question 3.2: (5 points)

The setting is the same as Question 1, so we do not have the additional constraints introduced in Question 2. However, the pollution tax is according to the interval approach, and we put a cap of 4000 Kgs on total pollution. Write the code for the variable pollution tax introduced in Question 3, and **print the total transportation cost, total pollution (in Kgs), and the total cost.** Note that in this problem, the total cost is the sum of the transportation costs and the variable tax.

In [35]:
Pollution = [[2, 4, 4, 5, 4, 5, 6, 2, 8, 2],
             [3, 10, 3, 4, 2.5, 2, 9, 1, 2, 5],
             [3, 4, 8, 3, 2, 1.5, 7, 5, 3, 8],
             [10, 5, 2, 5, 3, 5, 3, 6, 2, 7],
             [1, 1, 4, 5, 5, 2, 6, 2, 4, 3],
             [2, 4, 3, 4.5, 2, 7, 3, 3, 6, 1],
             [6, 7, 3, 3, 3, 6, 4, 6, 3, 10],
             [3, 3, 1, 6, 1, 5, 6, 3, 2, 4]]

Cost = [[35, 49, 16, 30,  15, 35, 12, 25, 10, 12],
        [15, 53, 7, 45, 47, 11, 16, 17, 15, 9],
        [22, 25, 42, 22, 31, 9, 11, 29, 36, 5],
        [32,  6, 40, 35, 49, 25, 30, 47, 32, 12],
        [9, 12, 19, 17, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 22, 5, 47, 30, 23, 19, 8],
        [43, 19,  5, 28, 47, 39, 15, 12, 9, 51],
        [34, 34, 10, 21, 9, 12, 14, 8, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [120, 100, 100, 260, 120, 250, 150, 500]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [100, 75, 150, 150, 180, 215, 170, 90, 160, 90]
Interval = [1000, 1750, 2500, 3250, 4000]
Tax = [0, 1000, 5000, 10000, 20000]

n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)

t = 3.5

##############################
# Initialize the model:
model = gb.Model("Question3")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Your code goes here:


# Create the decision variables:
x = model.addVars(n_supply, n_demand, vtype=GRB.CONTINUOUS, name='x')


y = model.addVars(n_demand, vtype=GRB.BINARY, name='y')

tax = model.addVar(vtype=GRB.CONTINUOUS, name='total tax')

# Introduce the objective function:
total_transport = sum(Cost[i][j] * x[i, j] for i in Range_supply for j in Range_demand)
total_pollution = sum(Pollution[i][j] * x[i, j] for i in Range_supply for j in Range_demand)




model.setObjective(total_transport + t * total_pollution + tax, GRB.MINIMIZE)

########################################################################################################################################################################################
########################################################################################################################################################################################

# Add the constraints:
model.addConstrs(quicksum(x[i,j] for j in Range_demand) <= Supply[i] for i in Range_supply)
model.addConstrs(quicksum(x[i,j] for i in Range_supply) == Demand[j] for j in Range_demand)

########################################################################################################################################################################################
########################################################################################################################################################################################

# Pollution constraints
model.addConstr(quicksum(Pollution[i][j] * x[i, j] for i in Range_supply for j in Range_demand) <= 4000)

########################################################################################################################################################################################
########################################################################################################################################################################################

# Constraints of Tax Calculation
M = 1000000
S = 0.0001

model.addConstr(tax == Tax[0] * y[0] + Tax[1] * y[1] + Tax[2] * y[2] + Tax[3] * y[3] + Tax[4] * y[4])
model.addConstr(total_pollution >= Interval[0] + S - M * y[0])
model.addConstr(total_pollution >= Interval[1] - M * (1 - y[0]))
model.addConstr(total_pollution >= Interval[1] + S - M * y[1])
model.addConstr(total_pollution >= Interval[2] - M * (1 - y[1]))
model.addConstr(total_pollution >= Interval[2] + S - M * y[2])
model.addConstr(total_pollution >= Interval[3] - M * (1 - y[2]))
model.addConstr(total_pollution >= Interval[3] + S - M * y[3])
model.addConstr(total_pollution >= Interval[4] - M * (1 - y[3]))
model.addConstr(total_pollution >= Interval[4] + S - M * y[4])
model.addConstr(y[0] + y[1] + y[2] + y[3] + y[4] == 1)


model.optimize()

########################################################################################################################################################################################
########################################################################################################################################################################################

# Print the optimal values of the objective functions:
print('Total Pollution = %g Kgs.' % sum(Pollution[i][j]*x[i,j].x for i in Range_supply for j in Range_demand))
print('Total Transport Cost = $%g.' % sum(Cost[i][j]*x[i,j].x for i in Range_supply for j in Range_demand))
print('Total Cost = $%g.' % model.objVal)

Total Pollution = 3700 Kgs.
Total Transport Cost = $15090.
Total Cost = $48040.


# Problem 2: Floor Planning (45 points)

## Problem Description
You are familiar with the floor planning problem from the assignment. (If not, you may review Problem 2 of Assignment 2.) In this problem, we want to solve a series of familiar and new questions.

Similar to the assignment, our goal is to find a floor plan that maximizes the total expected annual profit. Notice that the mall can have more than one unit of each kind of store/facility as long as you do not pass the maximum number limit.

## Data:
| Facility/Store | Expected Annual Profit (\$) | Occupied Area ($m^2$) | Max Number of Units |
| :--: | :--: | :--: | :--: |
| Restaurant-Type A | 500,000 | 600 | 5 |
| Restaurant-Type B | 250,000 | 200 | 6 |
| Clothing Store | 900,000 | 1,000 | 5 |
| Tech Store | 1,500,000 | 1,500 | 4 |
| Pharmacy | 700,000 | 700 | 3 |
| Department Store | 1,900,000 | 3,500 | 3 |
| Toy Store | 1,000,000 | 2,000 | 3 |
| Toilet | -50,000 | 250 | 3 |
| Security Division | -100,000 | 250 | 3 |

There are two fundamental constraints that you need to consider when designing the floor plan **in all questions**:

1. The total area of all stores and facilities should be at most 15,000 $m^2$.
2. The mall should have at least one complete toilet and one complete security division. (By "at least one complete," we mean $\geq 1$.)

Also, in all the questions, except Question 7, we assume there is no nonlinear *diminishing returns* effect. The profit coming from a type of store is simply the number of that store/facility multiplied by its expected annual profit. For example, if we have two Type A restaurants, their total profit is $2 \times 500,000 = \$1,000,000$.

Now, answer the following questions.

## Question 4: Integer Decision Variables (5 points)

Similar to Question 5 of the assignment, suppose the number of stores/facilities should be discrete. While respecting the two fundamental constraints, **find and report the optimal expected annual profit and the total area utilized.**

In [36]:
Type = ['Restaurant-Type A', 'Restaurant-Type B', 'Clothing Store', 'Tech Store', 'Pharmacy', 'Dept Store', 'Toy Store', 'Toilet', 'Security']

Profit = [500000, 250000, 900000, 1500000, 700000, 1900000, 1000000, -50000, -100000]

Area = [600, 200, 1000, 1500, 700, 3500, 2000, 250, 250]

Max_No = [5, 6, 5, 4, 3, 3, 3, 3, 3]

Num = len(Type)

###############################################
# Create a new model
model = gb.Model("Question4")
model.setParam('MIPGap', 0.00001)

# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:

########################################################################################################################################################################################
########################################################################################################################################################################################

# Create the decision variables:
X = model.addVars(Num, vtype = 'I', name = ['x_'+k for k in Type])


########################################################################################################################################################################################
########################################################################################################################################################################################

# Introduce the objective function:
model.setObjective(quicksum(Profit[i]*X[i] for i in range(Num)), GRB.MAXIMIZE)


########################################################################################################################################################################################
########################################################################################################################################################################################

# Add the constraints:
model.addConstr(quicksum(Area[i]*X[i] for i in range(Num)) <= 15000)
model.addConstr(X[7] >= 1)
model.addConstr(X[8] >= 1)
model.addConstrs(X[i] <= Max_No[i] for i in range(Num))


########################################################################################################################################################################################
########################################################################################################################################################################################

# Optimize the model:
model.optimize()

# Print the optimal values of the objective functions:
print(f'Total Profit = $ {round(model.objVal)} ')
print('Total Area Occupied = %g.' % sum(Area[i]*X[i].x for i in range(Num)))






Set parameter MIPGap to value 1e-05
Total Profit = $ 14050000 
Total Area Occupied = 15000.


## Question 5: Buying an additional piece of land (10 points)

You can buy an additional 5000 $m^2$ piece of land for \$1,000,000 per year and annex it to the mall. Would you do it? Why or why not? (Use integer decision variables for the number of stores/facilities.)

(This assumption is independent of the future questions. When answering the next questions, do not consider the possibility of buying that piece of land.)

In [37]:
Type = ['Restaurant-Type A', 'Restaurant-Type B', 'Clothing Store', 'Tech Store', 'Pharmacy', 'Dept Store', 'Toy Store', 'Toilet', 'Security']

Profit = [500000, 250000, 900000, 1500000, 700000, 1900000, 1000000, -50000, -100000]

Area = [600, 200, 1000, 1500, 700, 3500, 2000, 250, 250]

Max_No = [5, 6, 5, 4, 3, 3, 3, 3, 3]

Num = len(Type)

###############################################
# Create a new model
model = gb.Model("Question5")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:

# Create the decision variables:
X = model.addVars(Num, vtype = 'I', name = ['x_'+k for k in Type])

# buy more land or not
L = model.addVar(vtype = 'B', name = 'Buy more land or not')

# Introduce the objective function:
model.setObjective(quicksum(Profit[i]*X[i] for i in range(Num)) - L*1000000, GRB.MAXIMIZE)


########################################################################################################################################################################################
########################################################################################################################################################################################

# Add the constraints:
model.addConstr(quicksum(Area[i]*X[i] for i in range(Num)) <= 15000 + L*5000)
model.addConstr(X[7] >= 1)
model.addConstr(X[8] >= 1)
model.addConstrs(X[i] <= Max_No[i] for i in range(Num))


########################################################################################################################################################################################
########################################################################################################################################################################################

# Optimize the model:
model.optimize()

# Print the optimal values of the objective functions:
print(f'Value of L (Buy more land or not): {L.x}')
print(f"More land should be bought as the investment is leading to increased profits by {16450000 -14050000}")
print(f'Total Profit = $ {round(model.objVal)} ')
print('Total Area Occupied = %g.' % sum(Area[i]*X[i].x for i in range(Num)))






Set parameter MIPGap to value 1e-05
Value of L (Buy more land or not): 1.0
More land should be bought as the investment is leading to increased profits by 2400000
Total Profit = $ 16450000 
Total Area Occupied = 19800.


*Write your suggestion here.*

More land should be bought as the investment is leading to increased profits by 2400000. Obviously this after removing the investment cost from profits

Total Profit = $ 16450000 

Total Area Occupied = 19800.


## Question 6: Some Additional Constraints

Similar to Question 6 of the assignment, we want to add some additional constraints to the problem - on top of the two fundamental constraints. In this question, we want to add the following constraints to the setup of Question 4 where we used integer decision variables:
1. The number of Type B restaurants cannot be more than the number of Type A restaurants.
2. The total number of security divisions should be at least half of the total number of department stores and tech stores combined.
3. We can have a pharmacy (or multiple pharmacies) only if we have at least one restaurant (regardless of the type of restaurant). 

(Note that these constraints are for this question only. So, do not consider them in Question 7.)

### Question 6.1: (10 points)
Write the mathematical formulation of each of the three constraints above. (Use $x_i$ to denote the number of stores/facilities of type $i$. Similar to Python, start the indices from 0. Therefore, we have: \
$x_0$: Number of Type A restaurants \
$x_1$: Number of Type B restaurants \
$x_2$: Number of clothing stores \
$x_3$: Number of tech stores \
$x_4$: Number of pharmacies \
$x_5$: Number of department stores \
$x_6$: Number of toy stores \
$x_7$: Number of toilets \
$x_8$: Number of security divisions


### First Constraint: The number of Type B restaurants cannot be more than the number of Type A restaurants.

x_1 <= x_0 
...

### Second Constraint: The total number of security divisions should be at least half of the total number of department stores and tech stores combined.

2*x_8 >= x_5 + x_3
...

### Third Constraint: We can have a pharmacy (or multiple pharmacies) only if we have at least one restaurant (regardless of the type of restaurant). 

...

x_4 <= x_0 + x_1 - 1  

### Question 6.2: (5 points)

Solve the problem with the three additional constraints above. **Find and report the optimal expected annual profit and the total area utilized.**

In [38]:
Type = ['Restaurant-Type A', 'Restaurant-Type B', 'Clothing Store', 'Tech Store', 'Pharmacy', 'Dept Store', 'Toy Store', 'Toilet', 'Security']

Profit = [500000, 250000, 900000, 1500000, 700000, 1900000, 1000000, -50000, -100000]

Area = [600, 200, 1000, 1500, 700, 3500, 2000, 250, 250]

Max_No = [5, 6, 5, 4, 3, 3, 3, 3, 3]

Num = len(Type)

###############################################
# Create a new model
model = gb.Model("Question6")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:

# Create the decision variables:
X = model.addVars(Num, vtype = 'I', name = ['x_'+k for k in Type])

# Introduce the objective function:
model.setObjective(quicksum(Profit[i]*X[i] for i in range(Num)), GRB.MAXIMIZE)

########################################################################################################################################################################################
########################################################################################################################################################################################

# Add the constraints:
model.addConstr(quicksum(Area[i]*X[i] for i in range(Num)) <= 15000)
model.addConstr(X[7] >= 1)
model.addConstr(X[8] >= 1)
model.addConstrs(X[i] <= Max_No[i] for i in range(Num))

## CONSTRAINTS FOR THIS QUESTION:
### First Constraint: The number of Type B restaurants cannot be more than the number of Type A restaurants.
model.addConstr(X[1] <= X[0])


### Second Constraint: The total number of security divisions should be at least half of the total number of department stores and tech stores combined.

########################################################################################################################################################################################
########################################################################################################################################################################################


model.addConstr(2*X[8] >= X[5] + X[3])
...

### Third Constraint: We can have a pharmacy (or multiple pharmacies) only if we have at least one restaurant (regardless of the type of restaurant). 

model.addConstr(X[4] <= X[0] + X[1] - 1)

# Optimize the model:
model.optimize()

# Print the optimal values of the objective functions:
print(f'Total Profit = $ {round(model.objVal)} ')
print('Total Area Occupied = %g.' % sum(Area[i]*X[i].x for i in range(Num)))


Set parameter MIPGap to value 1e-05
Total Profit = $ 13400000 
Total Area Occupied = 14850.


## Question 7: Using the Corridors for the Cafeteria (15 points)

As you have seen in many malls, there can be some cafeterias in the corridors. In this question, we want to know if we should add cafeteria(s) to our mall. Our mall has 1,000 $m^2$ of shared space in the corridors. Notice that this shared space is independent of that 15,000 $m^2$ of the floorplan. That means adding cafeteria(s) to the mall does not reduce the total area for other stores/facilities. Also, not having a cafeteria does not increase the total area for other stores/facilities.

Adding cafeteria(s) to the mall has multiple effects:
1. Each cafeteria requires 200 $m^2$ of the shared space. Since there is 1,000 $m^2$ shared space in the mall that can be allocated to cafeterias, we can have up to 5 cafeterias. However, there are some additional safety concerns and taxes due to the congestion in the corridors. The following table shows the number of cafeterias and the corresponding annual tax:

| Number of Cafeterias | Tax (\$) |
| :--: | :--: |
| 0 | 0 |
| 1 | 0 |
| 2 | 10,000 |
| 3 | 20,000 |
| 4 | 50,000 |
| 5 | 100,000 |

(Note that the above table shows the total tax for all cafeterias. For example, if we have 3 cafeterias, the total tax is 20,000 dollars, not 60,000 dollars.)

2. Due to the diminishing return to scale, the total profit from cafeterias is not simply the number of cafeterias multiplied by the expected annual profit of each cafeteria. Instead, the total profit coming from cafeterias is in the following form:

| Number of Cafeterias | Total Profit (\$) |
| :--: | :--: |
| 0 | 0 |
| 1 | 700,000 |
| 2 | 1,200,000 |
| 3 | 1,500,000 |
| 4 | 1,600,000 |
| 5 | 1,650,000 |

3. If the number of cafeterias is 3 or more, the number of Type B restaurants cannot be more than 2.

Now, with the possibility of adding cafeterias to the mall, and just with the two fundamental constraints, **find and print the optimal expected annual profit and the optimal number of cafeterias.** 

In [39]:
Type = ['Restaurant-Type A', 'Restaurant-Type B', 'Clothing Store', 'Tech Store', 'Pharmacy', 'Dept Store', 'Toy Store', 'Toilet', 'Security']

Profit = [500000, 250000, 900000, 1500000, 700000, 1900000, 1000000, -50000, -100000]

Area = [600, 200, 1000, 1500, 700, 3500, 2000, 250, 250]

Max_No = [5, 6, 5, 4, 3, 3, 3, 3, 3]

Num = len(Type)

Cf_Tax = [0, 0, 10000, 20000, 50000, 100000] 

Cf_Profit = [0, 700000, 1200000, 1500000, 1600000, 1650000]

###############################################
# Create a new model
model = gb.Model("Question7")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:


# Create the decision variables:
X = model.addVars(Num, vtype = 'I', name = ['x_'+k for k in Type])
C = model.addVars(len(Cf_Tax), vtype = 'B', name = ['c_'+str(k) for k in range(len(Cf_Tax))])

C_Tax = model.addVar(lb = 0, vtype = 'C', name = 'C_Tax') # auxilary variable to make life easy 
C_Profit = model.addVar(lb = 0, vtype = 'C', name = 'C_Profit') # auxilary variable to make life easy 
temp = model.addVar(lb =0, vtype = 'B', name = 'temp') # auxilary variable to make life easy

M =10000000000

# Introduce the objective function:
model.setObjective(quicksum(Profit[i]*X[i] for i in range(Num)) -C_Tax + C_Profit, GRB.MAXIMIZE)

# Add the constraints:
model.addConstr(quicksum(Area[i]*X[i] for i in range(Num)) <= 15000)
model.addConstr(X[7] >= 1)
model.addConstr(X[8] >= 1)
model.addConstrs(X[i] <= Max_No[i] for i in range(Num))

# Special constraints for this question


model.addConstr(sum(C[k] for k in range(len(Cf_Tax))) == 1)
model.addConstr(sum(Cf_Tax[k]*C[k] for k in range(len(Cf_Tax))) == C_Tax)
model.addConstr(sum(Cf_Profit[k]*C[k] for k in range(len(Cf_Tax))) == C_Profit)

model.addConstr(0 <= 2 - X[1] + M*temp)
model.addConstr(sum(C[k]*k for k in range(len(Cf_Tax))) - 2.999999999999999 <= 0 + M*(1-temp))




# Optimize the model:
model.optimize()

# Print the optimal values of the objective functions:
print(f'Total Profit = $ {round(model.objVal)} ')
print('Total Area Occupied = %g.' % sum(Area[i]*X[i].x for i in range(Num)))

print(f'optimal number of cafeteria is 3')



for k in range(len(Cf_Tax)):
    var_name = 'c_' + str(k)
    value = C[k].x
    print(f'{var_name}: {value}')

Set parameter MIPGap to value 1e-05
Total Profit = $ 15530000 
Total Area Occupied = 15000.
optimal number of cafeteria is 3
c_0: 0.0
c_1: 0.0
c_2: 0.0
c_3: 1.0
c_4: 0.0
c_5: 0.0


# Problem 3: Volunteer Assignment Problem (60 points)

## The (True) Story:
Between June 24th and 26th, the 2023 INFORMS MSOM Conference was held at Mont Royal Center, hosted by McGill's Desautels faculty of management. The conference was a great success, and the organizing committee was pleased with the outcome. However, the success of the conference was not a coincidence. The organizing committee had to work over the clock to make it happen. In this part, we want to solve a problem that the organizing committee of this conference (and any organizing team of any conference or exhibition) faces: **How to assign volunteers to different tasks?** Here, we model this situation as an allocation problem and solve it using Gurobi. 

**Note: In all the following questions, each volunteer can be assigned to at most one task on each day. However, a volunteer can be assigned to different types of tasks during the conference, as long as they are not on the same day.**

## Data:
In this problem, we have 10 candidate volunteers (all graduate students) and several tasks. Since volunteers are not professional conference organizers, their performance varies among different tasks. If a volunteer is assigned to a task with the required skills, they can execute the task more efficiently and reliably. Additionally, volunteers who participate in the conference can attend the talks and workshops for free. This means while the attendance fee for the conference is 500 dollars, the volunteers do not pay the fee, which means a potential loss of 500 dollars for the organizing committee for each volunteer. Therefore, it may not be optimal for the organizing committee to recruit all the candidates. 

## Question 8: The Simplest Case

In this question, we set aside the cost-benefit analysis of recruiting volunteers and\or optimizing their performance. So, we only focus on the assignment problem. We just want to assign volunteers tasks so that every task has the number of volunteers it requires (neither more nor less). In this case, we have the following data about the tasks: (A\V = Audiovisual)

| Task No. | Date | Task | Number of Volunteers Required |
| :--: | :--: | :--: | :--: |
| 0 | June 24th | Registration | 3 |
| 1 | June 24th | A\V Assistance | 2 |
| 2 | June 24th | Lunch-Dinner | 2 |
| 3 | June 25th | Registration | 2 |
| 4 | June 25th | A\V Assistance | 2 |
| 5 | June 25th | Lunch-Dinner | 3 |
| 6 | June 26th | Registration | 0 |
| 7 | June 26th | A\V Assistance | 2 |
| 8 | June 26th | Lunch-Dinner | 2 |

And the following information about the volunteers:

| Volunteer No. | Name |
| :--: | :--: |
| 0 | A |
| 1 | B |
| 2 | C |
| 3 | D |
| 4 | E |
| 5 | F |
| 6 | G |
| 7 | H |
| 8 | I |
| 9 | J |

Since we just want a feasible assignment, we do not need to consider the skills of volunteers and the skills required for each task. The only constraints are that each volunteer can be assigned to at most one task on a day and that every task should receive just enough volunteers.

Hint: As we only care about the feasibility of an assignment and not optimality, you can set objective function = 1 (similar to the Sudoku problem you've seen in the class.)

### Question 8.1: (10 points)

Write the mathematical formulation of this problem. 

$n_j$ is the number of volunteers required for task $j$.

$X_{i,j}$: a binary variable which equals to 1 if volunteer $i$ is assigned to task $j$, and 0 otherwise.


### Objective Function:

Maximize 1 

### Constraints:

#Sum of tasks assigned per volunteer per day  == 1 

this can be done with the following formulation

1. model.addConstrs(quicksum(x[i,j] for j in range(0, 3)) <= 1 for i in range(Num_Volunteers))

2. model.addConstrs(quicksum(x[i,j] for j in range(3, 6)) <= 1 for i in range(Num_Volunteers))

3. model.addConstrs(quicksum(x[i,j] for j in range(6, 9)) <= 1 for i in range(Num_Volunteers))

#Sum of volunteers per task should be exactly equal to what is required

model.addConstrs(quicksum(x[i,j] for i in range(0,9)) == Volunteers_Required[j] for j in range(Num_Tasks))




### Question 8.2: (5 points)

Complete the code and **print a feasible assignment**.

In [40]:


####
Tasks = ['Reg24', 'AV24', 'Din24', 'Reg25', 'AV25', 'Din25', 'Reg26', 'AV26', 'Din26']
Volunteers_Required = [3, 2, 2, 2, 2, 3, 0, 2, 2]
Volunteers = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

Num_Tasks = len(Tasks)
Num_Volunteers = len(Volunteers)

Registration_Skills = [8, 4, 10, 5, 7, 2, 6, 6, 3, 9]
AudioVisual_Skills = [10, 7, 5, 8, 7, 4, 7, 10, 9, 6]
Dining_Skills = [3, 4, 5, 5, 7, 10, 3, 5, 8, 6]

# Create a new model
model = gb.Model("Question9")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Create variables
x = model.addVars(Num_Volunteers, Num_Tasks, vtype=GRB.BINARY, name=["Volunteer " + str(Volunteers[i]) + " for task " + str(Tasks[j]) for i in range(Num_Volunteers) for j in range(Num_Tasks)])

# Introduce the objective function:
model.setObjective(1, GRB.MINIMIZE)

# Introduce the constraints that each task should have the exact number of volunteers it requires:
for j in range(Num_Tasks):
    model.addConstr(quicksum(x[i,j] for i in range(Num_Volunteers)) == Volunteers_Required[j], f"Task_{Tasks[j]}_Volunteer_Requirement")

# Constraints ensuring each volunteer can only be assigned to one task per day:
model.addConstrs(quicksum(x[i,j] for j in range(0, 3)) <= 1 for i in range(Num_Volunteers))
model.addConstrs(quicksum(x[i,j] for j in range(3, 6)) <= 1 for i in range(Num_Volunteers))
model.addConstrs(quicksum(x[i,j] for j in range(6, 9)) <= 1 for i in range(Num_Volunteers))

# Optimize model
model.optimize()

# Print optimal performance score
#print("Optimal performance score:", model.objVal)


# For each task, print the volunteers assigned to it:
for j in range(Num_Tasks):
    print('Task', Tasks[j], 'is assigned to', end = ' ')
    for i in range(Num_Volunteers):
        if x[i,j].x == 1:
            print(Volunteers[i], end = ' ')
    print()






### APPROACH 2

# Tasks = ['Reg24', 'AV24', 'Din24', 'Reg25', 'AV25', 'Din25', 'Reg26', 'AV26', 'Din26']

# Volunteers_Required = [3, 2, 2, 2, 2, 3, 0, 2, 2]

# Volunteers = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

# Num_Tasks = len(Tasks)
# Num_Volunteers = len(Volunteers)

# ###############################################
# # Create a new model
# model = gb.Model("Question8")
# model.setParam('MIPGap', 0.00001)
# # We ask Gurobi not to print too much on screen
# model.Params.OutputFlag = 0

# # Create variables
# x = model.addVars(Num_Volunteers, Num_Tasks, vtype=GRB.BINARY, name = ["Volunteer " + str(Volunteers[i]) + " for task " + str(Tasks[j]) for i in range(Num_Volunteers) for j in range(Num_Tasks)])

# ################################################
# # Your code here:
# # Introduce the objective function:

# model.setObjective(1, GRB.MINIMIZE)


# ################################################

# ## Each volunteer can only be assigned to one task on one day:
# model.addConstrs(quicksum(x[i,j] for j in range(0, 3)) <= 1 for i in range(Num_Volunteers))
# model.addConstrs(quicksum(x[i,j] for j in range(3, 6)) <= 1 for i in range(Num_Volunteers))
# model.addConstrs(quicksum(x[i,j] for j in range(6, 9)) <= 1 for i in range(Num_Volunteers))

# #Sum of volunteers per task should be exactly equal to what is required

# model.addConstrs(quicksum(x[i,j] for i in range(0,9)) == Volunteers_Required[j] for j in range(Num_Tasks))



# # Optimize model
# model.optimize()




Set parameter MIPGap to value 1e-05
Task Reg24 is assigned to D E J 
Task AV24 is assigned to C F 
Task Din24 is assigned to B H 
Task Reg25 is assigned to B C 
Task AV25 is assigned to H J 
Task Din25 is assigned to D F I 
Task Reg26 is assigned to 
Task AV26 is assigned to A F 
Task Din26 is assigned to D H 


## Question 9: When Performance Matters (10 points)

In this question, we want to consider the skillset of each volunteer when doing the assignment. In this case, we have the following scores for each volunteer-task pair:

| Volunteer No. | Name | Registration | A\V Assistance | Lunch-Dinner |
| :--: | :--: | :--: | :--: | :--: |
| 0 | A | 8 | 10 | 3 |
| 1 | B | 4 | 7 | 4 |
| 2 | C | 10 | 5 | 5 |
| 3 | D | 5 | 8 | 5 |
| 4 | E | 7 | 7 | 7 |
| 5 | F | 2 | 4 | 10 |
| 6 | G | 6 | 7 | 3 |
| 7 | H | 6 | 10 | 5 |
| 8 | I | 3 | 9 | 8 |
| 9 | J | 9 | 6 | 6 |

The tasks information table is the same as the previous question.

The meaning of the numbers is as follows: If volunteer $i$ is assigned to task $j$, the performance score is the number in the corresponding cell. For example, if volunteer $0$ is assigned to Registration on June 24th and A\V Assistance on June 25th and 26th, the performance score of this assignment is $8 + 10 + 10 = 28$. In this question, we want to find the assignment that maximizes the total performance score. Adjust your code from the previous question to solve this problem and **report the optimal performance score**.


In [48]:
Tasks = ['Reg24', 'AV24', 'Din24', 'Reg25', 'AV25', 'Din25', 'Reg26', 'AV26', 'Din26']
Volunteers_Required = [3, 2, 2, 2, 2, 3, 0, 2, 2]
Volunteers = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

Num_Tasks = len(Tasks)
Num_Volunteers = len(Volunteers)

Registration_Skills = [8, 4, 10, 5, 7, 2, 6, 6, 3, 9]
AudioVisual_Skills = [10, 7, 5, 8, 7, 4, 7, 10, 9, 6]
Dining_Skills = [3, 4, 5, 5, 7, 10, 3, 5, 8, 6]

# Create a new model
model = gb.Model("Question9")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Create variables
x = model.addVars(Num_Volunteers, Num_Tasks, vtype=GRB.BINARY, name=["Volunteer " + str(Volunteers[i]) + " for task " + str(Tasks[j]) for i in range(Num_Volunteers) for j in range(Num_Tasks)])

# Introduce the objective function:
objective = quicksum(quicksum(
    x[i,j] * (Registration_Skills[i] if "Reg" in Tasks[j] else (AudioVisual_Skills[i] if "AV" in Tasks[j] else Dining_Skills[i])) 
    for j in range(Num_Tasks)) for i in range(Num_Volunteers))
model.setObjective(objective, GRB.MAXIMIZE)



model.addConstrs(quicksum(x[i,j] for i in range(Num_Volunteers)) == Volunteers_Required[j] for j in range(Num_Tasks))


################################################

## Each volunteer can only be assigned to one task on one day:
model.addConstrs(quicksum(x[i,j] for j in range(0, 3)) <= 1 for i in range(Num_Volunteers))
model.addConstrs(quicksum(x[i,j] for j in range(3, 6)) <= 1 for i in range(Num_Volunteers))
model.addConstrs(quicksum(x[i,j] for j in range(6, 9)) <= 1 for i in range(Num_Volunteers))




# Optimize model
model.optimize()

# Print optimal performance score
print("Optimal performance score:", model.objVal)

# For each task, print the volunteers assigned to it:
for j in range(Num_Tasks):
    print('Task', Tasks[j], 'is assigned to', end = ' ')
    for i in range(Num_Volunteers):
        if x[i,j].x == 1:
            print(Volunteers[i], end = ' ')
    print()


### APPROACH 2
# Tasks = ['Reg24', 'AV24', 'Din24', 'Reg25', 'AV25', 'Din25', 'Reg26', 'AV26', 'Din26']

# Volunteers_Required = [3, 2, 2, 2, 2, 3, 0, 2, 2]

# Volunteers = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

# Num_Tasks = len(Tasks)
# Num_Volunteers = len(Volunteers)

# Reigstration_Skills = [8, 4, 10, 5, 7, 2, 6, 6, 3, 9]
# AudioVisual_Skills = [10, 7, 5, 8, 7, 4, 7, 10, 9, 6]
# Dining_Skills = [3, 4, 5, 5, 7, 10, 3, 5, 8, 6]
# ###############################################
# # Create a new model
# model = gb.Model("Question9")
# model.setParam('MIPGap', 0.00001)

# # We ask Gurobi not to print too much on screen
# model.Params.OutputFlag = 0

# # Create variables
# x = model.addVars(Num_Volunteers, Num_Tasks, vtype=GRB.BINARY, name = ["Volunteer " + str(Volunteers[i]) + " for task " + str(Tasks[j]) for i in range(Num_Volunteers) for j in range(Num_Tasks)])

# #Allocation Assignment Performance Score
# # Registration_Perforamance = quicksum(x[i,j]*Reigstration_Skills[i] for i in range(Num_Volunteers)) 
# # AudioVisual_Performance = quicksum(x[i,j]*AudioVisual_Skills[i] for i in range(Num_Volunteers))
# # Dining_Performance = quicksum(x[i,j]*Dining_Skills[i] for i in range(Num_Volunteers))

# #Allocation Assignment Performance Score by Volunteer

# x_score = model.addVars(Num_Volunteers, vtype = GRB.CONTINUOUS, name = ["Score of Volunteer {i}" for i in range(Num_Volunteers)])
# model.addConstrs(x_score[i] == (quicksum(x[i,j]*Reigstration_Skills[i] for j in [0,3,6])  + quicksum(x[i,j]*AudioVisual_Skills[i] for j in [1,4,7])  + quicksum(x[i,j]*Dining_Skills[i] for j in [2,5,8])) for i in range(Num_Volunteers) )

# # Registration_Perforamance = quicksum(x[i,j]*Reigstration_Skills[i] for i in range(Num_Volunteers)) 
# # AudioVisual_Performance = quicksum(x[i,j]*AudioVisual_Skills[i] for i in range(Num_Volunteers))
# # Dining_Performance = quicksum(x[i,j]*Dining_Skills[i] for i in range(Num_Volunteers))




# # Introduce the objective function:

# # model.setObjective(Registration_Perforamance + AudioVisual_Performance +Dining_Performance, GRB.MAXIMIZE)
# model.setObjective(sum(x_score[i] for i in range(Num_Volunteers)), GRB.MAXIMIZE)


# ################################################

# ## Each volunteer can only be assigned to one task on one day:
# model.addConstrs(quicksum(x[i,j] for j in range(0, 3)) <= 1 for i in range(Num_Volunteers))
# model.addConstrs(quicksum(x[i,j] for j in range(3, 6)) <= 1 for i in range(Num_Volunteers))
# model.addConstrs(quicksum(x[i,j] for j in range(6, 9)) <= 1 for i in range(Num_Volunteers))

# #Sum of volunteers per task should be exactly equal to what is required

# model.addConstrs(quicksum(x[i,j] for i in range(0,9)) == Volunteers_Required[j] for j in range(Num_Tasks))



# # Optimize model
# model.optimize()

# print(f"Optimal performance score is {model.objVal}")

# # For each task, print the volunteers assigned to it:
# for j in range(Num_Tasks):
#     print('Task', Tasks[j], 'is assigned to', end = ' ')
#     for i in range(Num_Volunteers):
#         if x[i,j].x == 1:
#             print(Volunteers[i], end = ' ')
#     print()


Set parameter MIPGap to value 1e-05
Optimal performance score: 166.0
Task Reg24 is assigned to C E J 
Task AV24 is assigned to A H 
Task Din24 is assigned to F I 
Task Reg25 is assigned to C J 
Task AV25 is assigned to A H 
Task Din25 is assigned to E F I 
Task Reg26 is assigned to 
Task AV26 is assigned to A H 
Task Din26 is assigned to F I 


## Question 10: When Performance Matters and Volunteers are Expensive (10 points)

In this question, we want to consider the cost of recruiting volunteers. As mentioned in the problem description, each volunteer costs the organizing committee \$500. Either a volunteer is recruited (and the organization committee endures the cost) and can be assigned tasks on those three days, or they are not recruited (and the organization committee does not pay the cost) and cannot be assigned to any task.
Therefore, we want to find the assignment that maximizes the total performance score while minimizing the total cost of recruiting volunteers. Model this as a hierarchical multi-objective optimization problem, where the top priority is minimizing the recruiting cost, and the second priority is maximizing the total performance score. Adjust your code from Question 9 to solve this problem. **Report the optimal recruiting cost and the optimal performance score.**

(Hint: In Gubori's multi-objective optimization, all objectives' nature (maximization or minimization) should be the same. For this question, you can model maximizing the total performance score as minimizing the negative of the total performance score.)

In [42]:
Tasks = ['Reg24', 'AV24', 'Din24', 'Reg25', 'AV25', 'Din25', 'Reg26', 'AV26', 'Din26']
Volunteers_Required = [3, 2, 2, 2, 2, 3, 0, 2, 2]
Volunteers = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

Num_Tasks = len(Tasks)
Num_Volunteers = len(Volunteers)

Registration_Skills = [8, 4, 10, 5, 7, 2, 6, 6, 3, 9]
AudioVisual_Skills = [10, 7, 5, 8, 7, 4, 7, 10, 9, 6]
Dining_Skills = [3, 4, 5, 5, 7, 10, 3, 5, 8, 6]

# Create a new model
model = gb.Model("Question10")
model.setParam('MIPGap', 0.00001)
model.Params.OutputFlag = 0

# Variables
x = model.addVars(Num_Volunteers, Num_Tasks, vtype=GRB.BINARY, name=["Volunteer " + str(Volunteers[i]) + " for task " + str(Tasks[j]) for i in range(Num_Volunteers) for j in range(Num_Tasks)])
y = model.addVars(Num_Volunteers, vtype=GRB.BINARY, name=["Recruit " + str(Volunteers[i]) for i in range(Num_Volunteers)])

# First Objective: Minimize recruiting cost
recruit_cost = 500
total_cost = quicksum(y[i] * recruit_cost for i in range(Num_Volunteers))

# Second Objective: Minimize the negative of performance
performance = quicksum(quicksum(
    x[i,j] * (Registration_Skills[i] if "Reg" in Tasks[j] else (AudioVisual_Skills[i] if "AV" in Tasks[j] else Dining_Skills[i])) 
    for j in range(Num_Tasks)) for i in range(Num_Volunteers))

# Setting objectives with their priority
model.setObjectiveN(total_cost, index = 0, priority = 1)  # Primary Objective
model.setObjectiveN(-performance, index =1, priority = 0)  # Secondary Objective

# Constraints for recruitment
for i in range(Num_Volunteers):
    model.addConstr(quicksum(x[i,j] for j in range(Num_Tasks)) <= 3*y[i])

# Constraints that each task should have the exact number of volunteers it requires:
for j in range(Num_Tasks):
    model.addConstr(quicksum(x[i,j] for i in range(Num_Volunteers)) == Volunteers_Required[j])

# Constraints ensuring each volunteer can only be assigned to one task per day:
model.addConstrs(quicksum(x[i,j] for j in range(0, 3)) <= 1 for i in range(Num_Volunteers))
model.addConstrs(quicksum(x[i,j] for j in range(3, 6)) <= 1 for i in range(Num_Volunteers))
model.addConstrs(quicksum(x[i,j] for j in range(6, 9)) <= 1 for i in range(Num_Volunteers))

model.ModelSense = GRB.MINIMIZE

model.optimize()

# Output the optimal results
print("Optimal recruiting cost: $", total_cost.getValue())
print("Optimal performance score:", performance.getValue())

Set parameter MIPGap to value 1e-05
Optimal recruiting cost: $ 3500.0
Optimal performance score: 166.0


## Question 11: When Performance Matters and Volunteers Want to Attend Talks: (10 points)

The volunteers are all graduate students. Therefore, some of them want to attend the talks and workshops. The organization committee generously allows the volunteers to pick one of the three days as their off day, and they can attend the talks and workshops on that day. The following table, which is the updated version of the previous table, shows the day that each volunteer wants to attend the talks and workshops - so their day off:

| Volunteer No. | Name | Registration | A\V Assistance | Lunch-Dinner | Day Off |
| :--: | :--: | :--: | :--: | :--: | :--: |
| 0 | A | 8 | 10 | 3 | June 26th |
| 1 | B | 4 | 7 | 4 | June 25th |
| 2 | C | 10 | 5 | 5 | June 24th |
| 3 | D | 5 | 8 | 5 | June 26th |
| 4 | E | 7 | 7 | 7 | June 25th |
| 5 | F | 2 | 4 | 10 | June 26th |
| 6 | G | 6 | 7 | 3 | June 24th |
| 7 | H | 6 | 10 | 5 | June 26th |
| 8 | I | 3 | 9 | 8 | June 25th |
| 9 | J | 9 | 6 | 6 | June 24th |

If recruited, volunteer $i$ cannot be assigned to any task on their day off. For example, if volunteer $0$ is recruited, they will not be assigned to any task on June 26th. Adjust your code from Question 9 (without considering the recruiting costs) to solve this problem. **Report the optimal performance score.**

In [43]:
Tasks = ['Reg24', 'AV24', 'Din24', 'Reg25', 'AV25', 'Din25', 'Reg26', 'AV26', 'Din26']

Volunteers_Required = [3, 2, 2, 2, 2, 3, 0, 2, 2]

Volunteers = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

Num_Tasks = len(Tasks)
Num_Volunteers = len(Volunteers)

Reigstration_Skills = [8, 4, 10, 5, 7, 2, 6, 6, 3, 9]
AudioVisual_Skills = [10, 7, 5, 8, 7, 4, 7, 10, 9, 6]
Dining_Skills = [3, 4, 5, 5, 7, 10, 3, 5, 8, 6]

Unavailable_tasks_dict = {0: [6, 7, 8], 
                         1: [3, 4, 5],
                         2: [0, 1, 2],
                         3: [6, 7, 8],
                         4: [3, 4, 5],
                         5: [6, 7, 8],
                         6: [0, 1, 2],
                         7: [6, 7, 8],
                         8: [3, 4, 5],
                         9: [0, 1, 2]}
###############################################
# Create a new model
model = gb.Model("Question11")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:


# Create variables
x = model.addVars(Num_Volunteers, Num_Tasks, vtype=GRB.BINARY, name=["Volunteer " + str(Volunteers[i]) + " for task " + str(Tasks[j]) for i in range(Num_Volunteers) for j in range(Num_Tasks)])

# Introduce the objective function:
objective = quicksum(quicksum(
    x[i,j] * (Registration_Skills[i] if "Reg" in Tasks[j] else (AudioVisual_Skills[i] if "AV" in Tasks[j] else Dining_Skills[i])) 
    for j in range(Num_Tasks)) for i in range(Num_Volunteers))
model.setObjective(objective, GRB.MAXIMIZE)

# Introduce the constraints that each task should have the exact number of volunteers it requires:
for j in range(Num_Tasks):
    model.addConstr(quicksum(x[i,j] for i in range(Num_Volunteers)) == Volunteers_Required[j], f"Task_{Tasks[j]}_Volunteer_Requirement")

# Constraints ensuring each volunteer can only be assigned to one task per day:
model.addConstrs(quicksum(x[i,j] for j in range(0, 3)) <= 1 for i in range(Num_Volunteers))
model.addConstrs(quicksum(x[i,j] for j in range(3, 6)) <= 1 for i in range(Num_Volunteers))
model.addConstrs(quicksum(x[i,j] for j in range(6, 9)) <= 1 for i in range(Num_Volunteers))


# Off day constraints: 

model.addConstrs( sum(x[i,j] for j in Unavailable_tasks_dict[i]) == 0 for i in range(Num_Volunteers))

# Optimize model
model.optimize()

# Print optimal performance score
print("Optimal performance score:", model.objVal)

# For each task, print the volunteers assigned to it:
for j in range(Num_Tasks):
    print('Task', Tasks[j], 'is assigned to', end = ' ')
    for i in range(Num_Volunteers):
        if x[i,j].x == 1:
            print(Volunteers[i], end = ' ')
    print()



Set parameter MIPGap to value 1e-05
Optimal performance score: 141.0
Task Reg24 is assigned to A B E 
Task AV24 is assigned to D H 
Task Din24 is assigned to F I 
Task Reg25 is assigned to C G 
Task AV25 is assigned to A H 
Task Din25 is assigned to D F J 
Task Reg26 is assigned to 
Task AV26 is assigned to G I 
Task Din26 is assigned to E J 


## Question 12: ``Some People just Want to Watch the World Burn.''

Humans are complicated creatures - especially graduate students. Some of them hate each other and don't want to work together, some like each other and want to work together, and some are indifferent. The following table shows the relationship between the volunteers:
1. Volunteer A accepts a task only if Volunteer B is not assigned to that task on that day.
2. Volunteers E and F are best friends, so on each day, they either work together on the same task or do not work on that day.
3. Volunteers G and H are enemies and want to avoid working together. However, they will agree to work together only if Volunteer B is also assigned to that task at that time.

### Question 12.1: (10 points)

Write the mathematical formulation of these constraints. 

### Constraint 1: Volunteer A accepts a task only if Volunteer B is not assigned to that task on that day.

for j in range(num_taks):
    model.addConstr(X[0,j] + X[1,j] == 1)
...

### Constraint 2: Volunteers E and F are best friends, so on each day, they either work together on the same task or do not work on that day.

...
for j in range(num_taks):
    model.addConstr(X[4,j] === X[5,j])


### Constraint 3: Volunteers G and H are enemies and want to avoid working together. However, they will agree to work together only if Volunteer B is also assigned to that task at that time.

let q be a binary auxilary variable 


model.addConstr(X[6,j] + X[7,j] - 1 <= M * (1-q) for j in range(num_taks))

model.addConstr(0 <= X[6,j] + X[7,j] + X[1] - 3 + M*q for j in range(num_taks))
...


### Question 12.2: (5 points)

Adjust your code from Question 11 to solve this problem. Is Question 11 + the constraints in Question 12 feasible? If infeasible, can you drop (ignore) one of the three constraints and make it feasible? Which one?

In [44]:
Tasks = ['Reg24', 'AV24', 'Din24', 'Reg25', 'AV25', 'Din25', 'Reg26', 'AV26', 'Din26']

Volunteers_Required = [3, 2, 2, 2, 2, 3, 0, 2, 2]

Volunteers = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

Num_Tasks = len(Tasks)
Num_Volunteers = len(Volunteers)

Registration_Skills = [8, 4, 10, 5, 7, 2, 6, 6, 3, 9]
AudioVisual_Skills = [10, 7, 5, 8, 7, 4, 7, 10, 9, 6]
Dining_Skills = [3, 4, 5, 5, 7, 10, 3, 5, 8, 6]

Unavailable_tasks_dict = {0: [6, 7, 8], 
                         1: [3, 4, 5],
                         2: [0, 1, 2],
                         3: [6, 7, 8],
                         4: [3, 4, 5],
                         5: [6, 7, 8],
                         6: [0, 1, 2],
                         7: [6, 7, 8],
                         8: [3, 4, 5],
                         9: [0, 1, 2]}

M = 1000000000
###############################################
# Create a new model
model = gb.Model("Question12")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:

# Create variables
x = model.addVars(Num_Volunteers, Num_Tasks, vtype=GRB.BINARY, name=["Volunteer " + str(Volunteers[i]) + " for task " + str(Tasks[j]) for i in range(Num_Volunteers) for j in range(Num_Tasks)])

# Introduce the objective function:
objective = quicksum(quicksum(
    x[i,j] * (Registration_Skills[i] if "Reg" in Tasks[j] else (AudioVisual_Skills[i] if "AV" in Tasks[j] else Dining_Skills[i])) 
    for j in range(Num_Tasks)) for i in range(Num_Volunteers))
model.setObjective(objective, GRB.MAXIMIZE)

# Introduce the constraints that each task should have the exact number of volunteers it requires:
for j in range(Num_Tasks):
    model.addConstr(quicksum(x[i,j] for i in range(Num_Volunteers)) == Volunteers_Required[j], f"Task_{Tasks[j]}_Volunteer_Requirement")

# Constraints ensuring each volunteer can only be assigned to one task per day:
model.addConstrs(quicksum(x[i,j] for j in range(0, 3)) <= 1 for i in range(Num_Volunteers))
model.addConstrs(quicksum(x[i,j] for j in range(3, 6)) <= 1 for i in range(Num_Volunteers))
model.addConstrs(quicksum(x[i,j] for j in range(6, 9)) <= 1 for i in range(Num_Volunteers))


# Off day constraints: 

model.addConstrs( sum(x[i,j] for j in Unavailable_tasks_dict[i]) == 0 for i in range(Num_Volunteers))

# Constraints from Q12.2


# ## Constraint 1: Volunteer A accepts a task only if Volunteer B is not assigned to that task on that day.

for j in range(Num_Tasks):
    model.addConstr(x[0,j] + x[1,j] == 1)


# ## Constraint 2: Volunteers E and F are best friends, so on each day, they either work together on the same task or do not work on that day.


for j in range(Num_Tasks):
    model.addConstr(x[4,j] == x[5,j])


# ## Constraint 3: Volunteers G and H are enemies and want to avoid working together. However, they will agree to work together only if Volunteer B is also assigned to that task at that time.
# q = model.addVar(vtype = 'B')

# for j in range(Num_Tasks):
#     model.addConstr(x[6,j] + x[7,j] - 1.5 <= M * (1-q))
#     model.addConstr(0 <= x[6,j] + x[7,j] + x[1,j] - 3 + M*q)


# Optimize model
model.optimize()

# Print optimal performance score
print("Optimal performance score:", model.objVal)

# For each task, print the volunteers assigned to it:
for j in range(Num_Tasks):
    print('Task', Tasks[j], 'is assigned to', end = ' ')
    for i in range(Num_Volunteers):
        if x[i,j].x == 1:
            print(Volunteers[i], end = ' ')
    print()



Set parameter MIPGap to value 1e-05


AttributeError: Unable to retrieve attribute 'objVal'

*Your answer goes here:*


yes model is infeasible with new constraints 


if we comment out only constraint 1, it still remains unfeasible

if we comment out only constraint 2, it still remains unfeasible

if we comment out only constraint 3, it still remains unfeasible

...